In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from pathlib import Path
from sklearn.utils import compute_class_weight
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

In [63]:
X_attacker = np.load(Path("ClosestCarCSVsModel2/np_attacker_noscale.npy"))
X_normal = np.load(Path("ClosestCarCSVsModel2/np_normal.npy"))
# X_attacker = torch.load(Path("ClosestCarCSVsModel2/df_attacker.pt"))
# X_normal = torch.load(Path("ClosestCarCSVsModel2/df_normal.pt"))

X_normal_split, p = train_test_split(X_normal, test_size=0.70, random_state=42, shuffle=True)
print(X_normal_split[0])
print(X_attacker[0])

[[ 2.3800218  -1.6631837  -1.3700132  ... -1.3872428  -0.92077583
   1.2700201 ]
 [ 2.0176327  -1.519206   -1.3714483  ... -1.2015635  -0.9226978
   1.2727458 ]
 [ 1.6549435  -1.3751091  -1.3712893  ... -1.0579337  -0.939978
   1.2972533 ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
[[ 3.6098101e+03  6.0798438e+03 -3.7024779e+00 ...  5.7648008e+03
  -2.0161171e-02 -3.4568848e+01]
 [ 3.6061121e+03  6.0477354e+03 -3.7055876e+00 ...  5.7748677e+03
  -3.7707724e-02  3.8905666e+01]
 [ 3.6024116e+03  6.0156064e+03 -3.6929963e+00 ...  5.8137134e+03
  -3.7663493e-02  3.8860031e+01]
 ...
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.

In [64]:
X_attacker.shape

(9583, 100, 12)

In [51]:
y = np.asarray([0]*X_normal_split.shape[0] + [1]*X_attacker.shape[0]).astype('float32')
X = np.concatenate((X_normal_split, X_attacker), axis=0)
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, shuffle=True)

(20969, 100, 12)
(20969,)


In [61]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Define the training and validation data
# X = [
#     [[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.1, 0.2, 0.3],
#      [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.1, 0.2, 0.3, 0.4],
#      [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.1, 0.2, 0.3, 0.4, 0.5],
#      [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
#      [0.5, 0.6, 0.7, 0.8, 0.9, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]],
#     [[0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.1, 0.2, 0.3, 0.4],
#      [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.1, 0.2, 0.3, 0.4, 0.5],
#      [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
#      [0.5, 0.6, 0.7, 0.8, 0.9, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7],
#      [0.6, 0.7, 0.8, 0.9, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]]
# ]

# y = [0, 1]

# X = torch.tensor(X, dtype=torch.long, device=device)
# y = torch.tensor(y, dtype=torch.long, device=device)

# Create the dataset and dataloader
dataset = TimeSeriesDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=2000, shuffle=True)

# Define the hyperparameters
input_dim = 12
hidden_dim = 64
num_layers = 2
num_classes = 2
lr = 1e-3
num_epochs = 10

# Initialize the model, optimizer, and loss function
model = TimeSeriesClassificationModel(input_dim, hidden_dim, num_layers, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    for i, (sequences, labels) in enumerate(dataloader):
        sequences = sequences.to(device)
        labels = labels.to(device)
        labels = labels.to(torch.long)
#         print(labels)
        # Forward pass
        outputs = model(sequences)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'Batch {i}')
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

Batch 0
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Epoch 1/10, Loss: 0.6905
Batch 0
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Epoch 2/10, Loss: 0.6865
Batch 0
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Epoch 3/10, Loss: 0.6718
Batch 0
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Epoch 4/10, Loss: 0.6865
Batch 0
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Epoch 5/10, Loss: 0.6964
Batch 0
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Epoch 6/10, Loss: 0.6708
Batch 0
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Epoch 7/10, Loss: 0.6955
Batch 0
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Epoch 8/10, Loss: 0.6942
Batch 0
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Epoch 9/10, Loss: 0.6982
Batch 0
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Epoch 10/10, Loss: 0.6946


In [62]:
dataset = TimeSeriesDataset(X_test, y_test)
dataloader = DataLoader(dataset, batch_size=1000, shuffle=True)

model.eval()
with torch.no_grad():
    for sequences, labels in dataloader:
        sequences = sequences.to(device)
        labels = labels.to(device)



        # Forward pass
        outputs = model(sequences)
        _, predicted = torch.max(outputs.data, 1)

        # Calculate accuracy
        total = labels.size(0)
        correct = (predicted == labels).sum().item()

    print(f"Accuracy: {100 * correct / total:.2f}%")




Accuracy: 54.02%


In [18]:
# Define the time series classification model
class TimeSeriesClassificationModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes):
        super().__init__()
        self.transformer = nn.Transformer(
            d_model=input_dim,
            nhead=4,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=hidden_dim
        )
        self.fc = nn.Linear(input_dim, num_classes)
        
    def forward(self, x):
        x = x.permute(1, 0, 2)  # [seq_len, batch_size, input_dim]
        output = self.transformer(x, x)  # [seq_len, batch_size, input_dim]
        output = output.permute(1, 0, 2)  # [batch_size, seq_len, input_dim]
        output = self.fc(output[:, -1, :])  # [batch_size, num_classes]
        return output

# Define a custom dataset for the time series data
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx]), torch.tensor(self.y[idx])

In [7]:
n_feats = 12

# define the model
model = Sequential()
model.add(Bidirectional(LSTM(64, input_shape=(None, n_feats)))) # "None" allows for varying input lengths
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
class_weights = compute_class_weight(class_weight = "balanced", classes= np.unique(y_train), y= y_train)
class_weights = dict(zip(np.unique(y_train), class_weights))

mc = ModelCheckpoint('best_model2.h5', monitor='val_loss', mode='min', save_best_only=True)

model.fit(X_train, y_train, 
          epochs=3000, 
          batch_size=2000, 
          class_weight=class_weights,
          validation_split=0.2.
          callbacks=[mc])
model.save('model2_V1.h5')
test_loss, test_acc = model.evaluate(X_test, y_test)

print(test_loss, test_acc)

(14049,)
217/217 [==============================] - 4s 19ms/step - loss: 0.6964 - accuracy: 0.5035
0.6963749527931213 0.5034682154655457


In [14]:
y_pred = model.predict(X_test, batch_size=2000)
y_pred[y_pred > 0.5] = 1
y_pred[y_pred <= 0.5] = 0
y_pred.flatten()

print('Accuracy:', accuracy_score(y_test, y_pred.flatten()))
print('Precision:', precision_score(y_test, y_pred.flatten()))

4/4 [==============================] - 1s 351ms/step
Accuracy: 0.5034682080924856
Precision: 0.46440281030444963


# Other Transformer

https://keras.io/examples/nlp/text_classification_with_transformer/

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.layers import TransformerBlock

# Prepare the data
input_shape = (10, 1)  # input shape of each time series sequence
num_classes = 2  # number of classes for classification

x_train = ...  # input time series data of shape (num_samples, input_shape)
y_train = ...  # labels for the input data, shape (num_samples,)

# Define the model
inputs = Input(shape=input_shape)
x = inputs

embedding_dim = 128
num_heads = 8
ff_dim = 128

# Time series embedding layer
x = TransformerBlock(embed_dim=embedding_dim, num_heads=num_heads, ff_dim=ff_dim)(x)
x = GlobalMaxPooling1D()(x)
x = Dropout(0.1)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.1)(x)

outputs = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=1e-4), metrics=["accuracy"])

# Train the model
y_train = tf.keras.utils.to_categorical(y_train)  # convert labels to one-hot encoded vectors
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_split=0.1)

# Transformer

https://keras.io/examples/timeseries/timeseries_transformer_classification/

In [ ]:
import numpy as np


def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)


root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

n_classes = len(np.unique(y_train))

idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

y_train[y_train == -1] = 0
y_test[y_test == -1] = 0


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)




In [ ]:
y_train.shape

In [ ]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)
